<a href="https://colab.research.google.com/github/ritwiks9635/Chatbot_uasing_HF/blob/main/Build_ChatBot_using_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Build ChatBot using Attention**

[Dataset](https://www.kaggle.com/datasets/saurabhprajapat/chatbot-training-dataset)

[Dataset2](https://www.kaggle.com/datasets/projjal1/human-conversation-training-data)

[Dataset3](https://www.kaggle.com/datasets/grafstor/simple-dialogs-for-chatbot)

[Embedding](https://www.kaggle.com/datasets/pkugoodspeed/nlpword2vecembeddingspretrained)

In [12]:
from zipfile import ZipFile
data = "/content/https:/www.kaggle.com/datasets/kausr25/chatterbotenglish/chatterbotenglish.zip"
with ZipFile(data,"r") as z:
  z.extractall("Chatbot_data/data")
  print("the data has been extracted ")

the data has been extracted 


In [13]:
import re
import yaml
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
dir_path = '/content/Chatbot_data/data'
files_list = os.listdir(dir_path + os.sep)

questions = []
answers = []
for filepath in files_list:
    stream = open(dir_path + os.sep + filepath, "rb")
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 2 :
            questions.append(con[0])
            replies = con[1 :]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con) > 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = []
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = []
for i in range(len(answers_with_tags)) :
    answers.append('' + answers_with_tags[i] + '')

In [15]:
with open("/content/chatbot dataset.txt") as f:
    for line in f:
        split_lines = line.split("\t")
        if len(split_lines[0]) > 2:
            questions.append(split_lines[0])
            answers.append(split_lines[1])

In [16]:
with open("/content/dialogs.txt") as f:
    for line in f:
        split_lines = line.split("\t")
        if len(split_lines[0]) > 2:
            questions.append(split_lines[0])
            answers.append(split_lines[1])
print(len(answers))

4854


In [17]:
df = pd.DataFrame({"query" : questions, "result" : answers})
df.tail()

,query,result
4849,that's a good question. maybe it's not old age.,are you right-handed?\n
4850,are you right-handed?,yes. all my life.\n
4851,yes. all my life.,you're wearing out your right hand. stop using...
4852,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.\n
4853,but i do all my writing with my right hand.,start typing instead. that way your left hand ...


In [18]:
df.isnull().sum()

query     0
result    0
dtype: int64

In [19]:
df.drop_duplicates(inplace  = True)

In [20]:
df.shape

(4851, 2)

In [21]:
exclude = set(string.punctuation)
remove_digits = str.maketrans('', '', string.digits)

In [22]:
def preprocess_questions_sentences(sent):
    '''Function to preprocess English Sentence'''
    sent = sent.lower()
    sent = sent.replace("'", '')  # Changed .sub() to .replace()
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits)

    sent = sent.strip()
    sent = re.sub(" +", " ", sent)
    return sent

def preprocess_answer_sentence(sent):
    if isinstance(sent, str):
        sent = sent.lower()
        sent = sent.replace("'", '')
        sent = ''.join(ch for ch in sent if ch not in exclude)
        sent = sent.translate(remove_digits)
        sent = sent.strip()
        sent = re.sub(" +", " ", sent)
        sent = "startseq " + sent + " endseq"
        return sent
    else:

        return sent

In [23]:
df['query'] = df['query'].apply(preprocess_questions_sentences)
df['result'] = df['result'].apply(preprocess_answer_sentence)

In [24]:
df.head()

,query,result
0,you get paid,startseq i have no need for money endseq
1,stock market,startseq buy low sell high endseq
2,stock market,startseq invest in yourself endseq
3,stock market,startseq why not just take everything to a cas...
4,stock market,startseq i wouldnt recommend buying on the mar...


In [25]:
max_length = 30

ques_sentences = df['query'].tolist()
ans_sentences = df['result'].tolist()

n = len(ques_sentences)
split_index1 = int(0.95 * n)
split_index2 = int(0.98 * n)

train_ques_sents = ques_sentences[:split_index2]
test_ques_sents = ques_sentences[split_index2:]

train_ans_sents = ans_sentences[:split_index2]
test_ans_sents = ans_sentences[split_index2:]

In [26]:
train_ques_sents = [str(sent) for sent in train_ques_sents]
train_ans_sents = [str(sent) for sent in train_ans_sents]

# Tokenize question sentences
ques_tokenizer = Tokenizer(oov_token='<OOV>')
ques_tokenizer.fit_on_texts(train_ques_sents)
ques_vocab_size = len(ques_tokenizer.word_index) + 1

# Tokenize answer sentences
ans_tokenizer = Tokenizer()
ans_tokenizer.fit_on_texts(train_ans_sents)
ans_vocab_size = len(ans_tokenizer.word_index) + 1

In [27]:
print(f"Question Vocab Size: {ques_vocab_size}\nAnswer Vocab Size: {ans_vocab_size}")

Question Vocab Size: 2558
Answer Vocab Size: 3341


In [28]:
ques_sequence = ques_tokenizer.texts_to_sequences(train_ques_sents)
ans_sequence = ans_tokenizer.texts_to_sequences(train_ans_sents)

ques_padding = pad_sequences(ques_sequence, maxlen = max_length, padding = "post")
ans_padding = pad_sequences(ans_sequence, maxlen = max_length, padding = "post")

In [29]:
train_dataset = tf.data.Dataset.from_tensor_slices((ques_padding[:split_index1], ans_padding[:split_index1]))
train_dataset = train_dataset.shuffle(buffer_size = split_index1).batch(32, drop_remainder = True)

valid_dataset = tf.data.Dataset.from_tensor_slices((ques_padding[split_index1:], ans_padding[split_index1:]))
valid_dataset = valid_dataset.batch(32, drop_remainder = True)

In [30]:
embeddings_dict = {}

# Open the GloVe embeddings file
with open('/content/glove.6B.100d.txt', 'r', encoding='utf-8') as file:
    # Iterate over each line in the file
    for line in tqdm(file, desc='Loading Glove Embedding'):
        # Split the line into words and their corresponding embedding vectors
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')

        # Store the word and its embedding vector in the dictionary
        embeddings_dict[word] = coefs

# Create embedding matrices
ques_embedding_matrix = np.zeros((ques_vocab_size, 100))
ans_embedding_matrix = np.zeros((ans_vocab_size, 100))

# Populate embedding matrices with GloVe vectors for words in the vocabulary
for word, index in ques_tokenizer.word_index.items():
    if word in embeddings_dict:
        ques_embedding_matrix[index] = embeddings_dict[word]

for word, index in ans_tokenizer.word_index.items():
    if word in embeddings_dict:
        ans_embedding_matrix[index] = embeddings_dict[word]

Loading Glove Embedding: 400000it [00:14, 27855.54it/s]


In [31]:
loss_obj = keras.losses.SparseCategoricalCrossentropy()

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_obj(real, pred)

    mask = tf.cast(mask, dtype = loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

##**Attention Mechanism**

Attention mechanism are a type of neural network layer that can be add to deep learning models. They allow the model to focus on specific parts of the input. This weights is typically based on the relevence of each part of the input to the task at hand.Similarly for the task of Neural Text Generation.


**Additive Attention:** This mechanism refers to Dzmitry Bahdanau's work title "Neural Machine Translation by Jointly Learning to Align and Translate". This, this tecnique is also known as Bahdanau attention. The additive attention is implemented as follows

- Addition of weeighted Decoder hidden state and Encoder hidden states

- Tanh function is applied over this addition
-  Output of step 2 is passed through another linear layer
- Softmax function is applied to calculate the normalized allignment attention scores
- Attention scores are applied to the encoder hidden state to indentify the importance of each source word with respect to the next output word


**Multiplicative Attention:** This method is proposed by Thang Luong in the work title 'Effective Approaches to Attention-based Neural Machine Translation'. It is build on top of additive attention (a.k.a Bahdanau attention). there are three scoring functions that we can choose from - dot, general , concat,(we have used dot in our code)

- Matrix multiplication perfomed between Encoder hidden states and Deocder hidden state
-  Softmax function is applied t calculate the normalized allignment attention scores
- Attention scores are applied to the Encoder hidden states to indentify the importance of each source word with respect to the next output word/

In [32]:
class BahdanauAttention(layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = layers.Dense(units)
        self.W2 = layers.Dense(units)
        self.V = layers.Dense(1)


    def call(self, s_hidden, h_hidden):
        s_hidden = tf.expand_dims(s_hidden, axis =1)
        score = self.V(tf.nn.tanh(self.W1(s_hidden) + self.W2(h_hidden)))
        attention_weights = tf.nn.softmax(score, axis = 1)
        context_vector = attention_weights * h_hidden
        context_vector = tf.expand_dims(tf.reduce_sum(context_vector, axis = 1), axis = 1)
        return context_vector, attention_weights

In [33]:
# Class LuongAttention
class  LuongAttention(layers.Layer):
    def __init__(self, units):
        super(LuongAttention, self).__init__()

    def call(self, s_hidden, h_hidden):
        score = tf.matmul(h_hidden, s_hidden, transpose_b =True)
        attention_weights = tf.nn.softmax(score, axis = 1)
        context_vector = attention_weights * h_hidden
        context_vector = tf.expand_dims(tf.reduce_sum(context_vector, axis =1), axis =1)
        return context_vector, attention_weights

###**Sequence2Sequence Model With Attention Layer**

**Encoder**

- It uses RNN layer and converts the input words to corresponding hidden vectors. Each vector represents the current word and the contex of the word. The encoder takes the input sequence, one token at a time, and uses an to update it's hidden state, which summarizes the information inthe input sequence. The finel hidden state (state_h in the code) of the encoder is then passed as the context vector to the decoder. Since we are using Bidirectional LSTM as our RNN, we will get 2 hidden ecoder states (forward_state_h and backward_state_h) and 2 cell states (forward_state_c and bacword_state_c). We concatenate them into one hidden and cell state (state_h, and state-c in the code)

**Decoder**

- It takes as input the hidden vector generated by the encoder, it's own hidden states, and the current word to preduce the next hidden vector and finally predict the next word. In out case, since it is a Bi LSTM, we also take the cell sate along with the hidden state. **When using the attention layer we first calculate the attention context vector by applying attention mechanism on the decoder hidden state and the encoder hidden states, once the context vector is preduced it is concatenated wirh the current word embeding and passed to the decoder to predict the next word. The decoder useds this new context vector and an initial hidden state to generate the output sequence, one token at a time. At each time step, the decoder uses the current hidden state, the attention alligned context vector, and the previous output token to generate a probability distribution over the possible next tokens. The token with the highest probability is then chosen as the output, and the process continues until the end of the output sequence (end) is reached.

In [34]:
class Encoder(keras.Model):

    def __init__(self, vocab_size, embedding_dims, units):
        super(Encoder, self).__init__()
        self.embedding = layers.Embedding(vocab_size, embedding_dims, mask_zero=True)
        self.lstm = layers.Bidirectional(layers.LSTM(units, return_sequences=True, return_state=True))

    def call(self, x):
        x = self.embedding(x)
        enc_output, forward_h, forward_c, backward_h, backward_c = self.lstm(x)
        state_h = layers.Concatenate()([forward_h, backward_h])
        state_c = layers.Concatenate()([forward_c, backward_c])

        return enc_output, state_h, state_c

In [35]:
class Decoder(keras.Model):

    def __init__(self, vocab_size, embedding_dims, units, use_additive=True):
        super(Decoder, self).__init__()
        self.units = units
        self.embedding = layers.Embedding(vocab_size, embedding_dims, mask_zero=True)
        self.lstm = layers.LSTM(units, return_sequences=True, return_state=True)
        self.attention = BahdanauAttention(units) if use_additive else LuongAttention(units)
        self.fc = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))

    def call(self, x, enc_output, state_h, state_c):
        x = self.embedding(x)
        context_vector, attn_weights = self.attention(state_h, enc_output)
        context_vector = layers.Concatenate(axis=-1)([context_vector, x])
        dec_output, dec_h, dec_c = self.lstm(context_vector, initial_state=[state_h, state_c])
        output = self.fc(dec_output)

        return output, dec_h, dec_c, attn_weights

In [36]:
encoder = Encoder(ques_vocab_size, 512, 256)
decoder = Decoder(ans_vocab_size, 512, 512)

In [37]:
optimizer = keras.optimizers.Adam()

##**Teacher Forcing**

In [38]:
@tf.function
def train_step(x, y):
    loss_value = 0
    with tf.GradientTape() as tape:
        encoder_output, state_h, state_c  = encoder(x)
        dec_input = tf.expand_dims(y[:, 0], 1)

        for i in range(1, y.shape[1]):
            dec_output, state_h, state_c, attn_w = decoder(dec_input, encoder_output, state_h, state_c)
            loss_value += loss_function(y[:, i], dec_output[:, 0, :])
            dec_input = tf.expand_dims(y[:, i], 1)

    batch_loss = (loss_value / int(y.shape[1]))
    weights = encoder.trainable_variables + decoder.trainable_variables
    grads = tape.gradient(loss_value, weights)
    optimizer.apply_gradients(zip(grads, weights))
    return batch_loss

@tf.function
def test_step(x, y):
    loss_value = 0
    encoder_output, state_h, state_c  = encoder(x)
    dec_input = tf.expand_dims(y[:, 0], 1)

    for i in range(1, y.shape[1]):
        dec_output, state_h, state_c, attn_w = decoder(dec_input, encoder_output, state_h, state_c)
        loss_value += loss_function(y[:, i], dec_output[:, 0, :])
        dec_input = tf.expand_dims(y[:, i], 1)

    batch_loss = (loss_value / int(y.shape[1]))
    return batch_loss

In [ ]:
from tensorflow.keras.utils import Progbar


epochs = 10
train_steps = np.sum([1 for i in train_dataset])
val_steps = np.sum([1 for i in valid_dataset])  # Calculate number of validation steps
metrics_names = ['train_loss', 'val_loss']  # Include 'val_loss' in metrics_names
mon_val_loss = np.inf

for epoch in range(epochs):
    print("Epoch: ", epoch+1)
    training_loss = []
    validation_loss = []
    progBar = Progbar(train_steps, stateful_metrics=metrics_names)

    # Training loop
    for step, (X_batch_train, y_batch_train) in enumerate(train_dataset):
        train_loss = train_step(X_batch_train, y_batch_train)
        training_loss.append(train_loss)
        values = [('train_loss', train_loss)]
        progBar.update(step, values=values)

    avg_train_loss = np.average(training_loss)
    values = [('train_loss', avg_train_loss)]
    progBar.update(step, values)

    # Validation loop
    for step, (X_batch_val, y_batch_val) in enumerate(valid_dataset):
        val_loss = test_step(X_batch_val, y_batch_val)
        validation_loss.append(val_loss)

    avg_val_loss = np.average(validation_loss)
    values.append(('val_loss', avg_val_loss))
    progBar.update(train_steps, values=values, finalize=True)

    # Check if validation loss improved
    if avg_val_loss < mon_val_loss:
        encoder.save_weights("encoder.weights.h5")
        decoder.save_weights("decoder.weights.h5")
        print(f"Model loss improved from {mon_val_loss:.5f} to {avg_val_loss:.5f} Checkpoint Created: encoder.weights.h5, decoder.weights.h5")

    mon_val_loss = avg_val_loss

In [40]:
encoder.save_weights(" /content/encoder.weights.h5")
decoder.save_weights( "/content/decoder.weights.h5")

In [41]:
encoder.load_weights('/content/encoder.weights.h5')
decoder.load_weights('/content/decoder.weights.h5')

In [42]:
def predict_sentence(en_input):
    eng_sequence = ques_tokenizer.texts_to_sequences([en_input])
    en_input = pad_sequences(eng_sequence, maxlen=max_length, padding='post')
    hidden_state, next_h, next_c = encoder(en_input)
    attn_plot = []

    curr_token = np.zeros((1,1))
    curr_token[0,0] = ans_tokenizer.word_index['startseq']
    pred_sentence = ''

    for i in range(max_length):
        output, next_h, next_c, attn_w = decoder(curr_token, hidden_state, next_h, next_c)
        attn_plot.append(attn_w.numpy().reshape(-1,))
        next_token = np.argmax(output[:, 0, :], axis=1)[0]
        next_word = ans_tokenizer.index_word[next_token]
        if next_word == 'endseq':
            break
        else:
            pred_sentence += ' ' + next_word
            curr_token[0,0] = next_token

    return pred_sentence.strip()

In [43]:
while True:
    user_input = input("You: ")

    # Exit loop if user enters 'exit'
    if user_input.lower() == 'exit':
        print("Goodbye!")
        break

    # Get response from the model
    response = predict_sentence(user_input)

    print("User Input:", user_input)
    print("Chatbot:", response)

You: What is ai
User Input: What is ai
Chatbot: ai is the field of science which concerns itself with the branch of engineering and science devoted to constructing machines in surgery etc
You: Whai is stock
User Input: Whai is stock
Chatbot: im not sure how much is an stock
You: Hi
User Input: Hi
Chatbot: hello
You: how are you
User Input: how are you
Chatbot: i am not into violence
You: I am fine and you
User Input: I am fine and you
Chatbot: i dont know you knew
You: You get paid
User Input: You get paid
Chatbot: i certainly do
You: Stock market
User Input: Stock market
Chatbot: mutual funds might be better unless you are wealthy
You: Stock merket
User Input: Stock merket
Chatbot: you can be better unless you
You: Exit
Goodbye!
